# LabOP Labware development

Why using a EMMOntoPy base approach

- the ontology is formulated programmatically in python
- supports developer with a lot of automatic functionality, like using UUIDs as IDs (by default), SKOS-labels (prefered label) , annotations, ...
- includes dimension and unit support out of the box (no extra definition required) - also many other concepts that are useful 
- unifies the way, the OWL based ontology is generated
- has verification tool to check the syntactic consistency of the ontology
- fast FACT++ reasoner for logic consistency check
- easy generation of turtle (ttl) OWL files and many other formats
- basic query is directly supported
- integrated SPRAQL engine for advance queries

 - everything is encapsulated in python classes, that can directly be used in applications (!!)


## Design guidelines for the labware ontology

- as general as possible -> wide applicability
- capturing the most important/common labware features
- separation of abstract class definitions and individuals
- no hard coded features

 ## outlook

 - improved properties / relations (proper auto - unit support)
 - packing everything into a nice package
 - copying code to LapOP repository (best in a new project)

In [ ]:
#  currently only reflib 4.2.1 is supported
#! pip install rdflib==4.2.1
#! pip install packaging==21.0
import os
import pathlib
import logging
import pandas as pd

from ontopy import World
from ontopy.utils import write_catalog

import owlready2
from owlready2 import DatatypeProperty, FunctionalProperty

# --- helper functions

def en(s):
    """Returns `s` as an English location string."""
    return owlready2.locstr(s, lang='en')


def pl(s):
    """Returns `s` as a plain literal string."""
    return owlready2.locstr(s, lang='')

In [ ]:
class labop_Labware:
    def __init__(self, emmo_world=None) -> None:

        __version__ = "0.0.1"
        __file__ = "."

        self.__version__ = __version__

        self.labop_labware_base_iri = 'http://www.oso.org/oso/labware#'
        self.labop_labware_version_iri = f'http://www.oso.org/{__version__}/oso/labware'

        output_filename_base = os.path.join('labop_labware')
        self.labop_labware_owl_filename = f'{output_filename_base}-v{__version__}.owl'
        self.labop_labware_ttl_filename = f'{output_filename_base}-v{__version__}.ttl'

        # Load crystallography, which imports emmo
        # alternative url   "https://raw.githubusercontent.com/emmo-repo/EMMO/master/self.emmo.ttl"

        self.emmo_url = (
            'https://raw.githubusercontent.com/emmo-repo/emmo-repo.github.io/'
            'master/versions/1.0.0-beta/emmo-inferred-chemistry2.ttl')

        self.emmo_url_local = os.path.join(pathlib.Path(
            __file__).parent.resolve(), "emmo", "emmo-inferred-chemistry2")

        if os.path.isfile(self.emmo_url_local + '.ttl'):
            self.emmo_url = self.emmo_url_local

        #self.emmo_world = World(filename="emmo_labwares.sqlite3")
        if emmo_world is not None:
            self.emmo_world = emmo_world
        else:
            self.emmo_world = World()
            # self.emmo_world.onto_path.append("../emmo")

            self.emmo = self.emmo_world.get_ontology(self.emmo_url)
            self.emmo.load()  # reload_if_newer = True
            self.emmo.sync_python_names()  # Synchronise annotations
            self.emmo.base_iri = self.emmo.base_iri.rstrip('/#')
            self.catalog_mappings = {self.emmo.base_iri: self.emmo_url}

        # Create new ontology
        self.osolw = self.emmo_world.get_ontology(self.labop_labware_base_iri)
        if emmo_world is None:
            self.osolw.imported_ontologies.append(self.emmo)
        self.osolw.sync_python_names()

    def define_ontology(self):
        logging.debug('defining labware ontology')

        with self.osolw:

            # Basic Relations
            # ================

            class hasType(self.osolw.hasConvention):
                """Associates a type (string, number...) to a property."""

            class isTypeOf(self.osolw.hasConvention):
                """Associates a property to a type (string, number...)."""
                inverse_property = hasType

            # Physical Properties
            # ==========

            class Length:
                """"Labware total length """

            class Width:
                """Labware total width, """
            
            class Height:
                """Labware total hight, without  any additions, like lids etc. """

            class Volume:
                """Total Labware volume """

            class HigthLidded:
                """Labware total hight, with additions, like lids etc."""

            class HightStacked:
                """Labware stacking height without any additions, like lids."""

            class HightStackedLidded:
                """Labware stacking height with additions, like lids."""

            class Mass:
                """Mass of the Labware """

            class Material:
                """Polymer, properties, like solvent tolerance, transparency, ...."""

            class Model3D:
                """3D model of the labware in X format."""
            
            class Color:
                """Labware color"""

            class Liddable:
                """container is liddable"""

            class Sealable:
                """container is sealable"""

            # multiwell labware

            class WellVolume:
                """Total Labware volume """

            class WellDistRow:
                """wWll-to-well distance in row direction"""
            
            class WellDistCol:
                """"Well-to-well distance in column direction"""

            # Well properties of labware with wells
            class DepthWell:
                """Well total well depth=hight"""
            
            class ShapeWell:
                """Well overall / top well shape,e.g. round, square, buffeled,..."""
            
            class ShapeWellBottom:
                """Well, bottom shape, flat, round, conical-"""

            class TopRadiusXY:
                """Well radius of a round well at the top opening in x-y plane."""

            class BottomRadiusXY:
                """Radius of a round bottom in xy plane / direction."""

            class BottomRadiusZ:
                """Radius of a round bottom in z (hight) direction."""

            class ConeAngle:
                """Opening angle of cone in deg."""

            class ConeDepth:
                """Depth of cone from beginning of conical shape."""

            class ShapePolygonXY:
                """Generalized shape polygon for more complex well shapes, in xy plane / direction."""

            class ShapePolygonZ:
                """Generalized shape polygon for more complex well shapes, in z direction = rotation axis."""

            class ShapeModel2D:
                """2D model of Well shape"""

            class ShapeModel3D:
                """3D model of Well shape"""



            # class Description:
            #     """Labware description. Possible applications/purpose for this labware could be also added here."""
                

            # Labware Classes
            # ====================

            # Basic ------

            class Labware(self.osolw.Device):
                """Labware is a utility device that all experiments are done with and which is not actively measuring. Examples: a container, a pipette tip, a reactor, ... """

                # is_a = [self.osolw.has_Material.some(str),
                #         self.osolw.has_NumCols.some(int),
                #         self.osolw.has_NumRows.some(int)]

            #  Relations / Properties
            # ========================

            # Physical Properties

            class hasLength:
                """"Labware total length """
                is_a = [
                    self.osolw.hasReferenceUnit.only(
                        self.osolw.hasPhysicalDimension.only(self.osolw.Length)
                    ),
                    hasType.exactly(1, self.osolw.Real), ]

            # class hasWidth(FunctionalProperty):
            #     """Labware total width, """
            #     domain = [Labware]
            #     range = [Width]

            # class hasHeight(Labware >> self.osolw.Height, FunctionalProperty):
            #     """Labware total hight, without  any additions, like lids etc. """
            
            class hasWidth(Labware >> float, FunctionalProperty):
                """Labware total width, """
            
            class hasHeight(Labware >> float, FunctionalProperty):
                """Labware total hight, without  any additions, like lids etc. """

            class hasRadiusXY(Labware >> float, FunctionalProperty):
                """Labware radius of a round shape in XY direction """

            class hasRadiusZ(Labware >> float, FunctionalProperty):
                """Labware radius of a round shape in XY direction """

            class hasVolume(Labware >> float, FunctionalProperty):
                """Total Labware volume """

            class hasHigthLidded(Labware >> float, FunctionalProperty):
                """Labware total hight, with additions, like lids etc."""

            class hasHightStacked(Labware >> float, FunctionalProperty):
                """Labware stacking height without any additions, like lids."""

            class hasHightStackedLidded(Labware >> float, FunctionalProperty):
                """Labware stacking height with additions, like lids."""

            class hasMass(Labware >> float, FunctionalProperty):
                """Mass of the Labware """

            
            class hasColor(Labware >> str, FunctionalProperty):
                """Labware color in RGB hex encoding"""

            class isLiddable(Labware >> bool, FunctionalProperty):
                """container is liddable"""

            class isSealable(Labware >> bool, FunctionalProperty):
                """container is sealable"""

            class hasMaterial(Labware >> str, DatatypeProperty):
                """Polymer, properties, like solvent tolerance, transparency, ...."""
                domain = [Labware]
                range = [str]

            # multiwell labware
            

            class hasNumCols(Labware >> int, FunctionalProperty):
                """Number of Columns of muti-well labware"""

            class hasNumRows(Labware >> int, FunctionalProperty):
                """Number of Rows of Labware"""

            class hasNumWells(Labware >> int, FunctionalProperty):
                """Number of Wells of muti-well labware"""

            # Production Properties / Metadata

            class hasManifacturer(Labware >> str, FunctionalProperty):
                 """Name of the Manufacturer """
            
            class isProductType(Labware >> str, FunctionalProperty):
                """Labware product Type"""

            class hasModelNumber(Labware >> str, FunctionalProperty):
                """Labware model number"""

            class hasProductNumber(Labware >> str, FunctionalProperty):
                """Manufacturer Product Number of the Labware"""

            # multiwell labware

            class hasWellVolume(Labware >> float, FunctionalProperty):
                """Total Labware volume """

            class hasWellDistRow(Labware >> float, FunctionalProperty):
                """wWll-to-well distance in row direction"""
            
            class hasWellDistCol(Labware >> float, FunctionalProperty):
                """"Well-to-well distance in column direction"""

            # Well properties of labware with wells
            class hasDepthWell(Labware >> float, FunctionalProperty):
                """Well total well depth=hight"""
            
            class hasShapeWell(Labware >> str, FunctionalProperty):
                """Well overall / top well shape,e.g. round, square, buffeled,..."""
            
            class hasShapeWellBottom(Labware >> str, FunctionalProperty):
                """Well, bottom shape, flat, round, conical-"""

            class hasTopRadiusXY(Labware >> float, FunctionalProperty):
                """Well radius of a round well at the top opening in x-y plane."""

            class hasBottomRadiusXY(Labware >> float, FunctionalProperty):
                """Radius of a round bottom in xy plane / direction."""

            class hasBottomRadiusZ(Labware >> float, FunctionalProperty):
                """Radius of a round bottom in z (hight) direction."""

            class hasConeAngle(Labware >> float, FunctionalProperty):
                """Opening angle of cone in deg."""

            class hasConeDepth(Labware >> float, FunctionalProperty):
                """Depth of cone from beginning of conical shape."""

            class hasShapePolygonXY(Labware >> float, FunctionalProperty):
                """Generalized shape polygon for more complex well shapes, in xy plane / direction."""

            class hasShapePolygonZ(Labware >> str, FunctionalProperty):
                """Generalized shape polygon for more complex well shapes, in z direction = rotation axis."""

            class hasShapeModel2D(Labware >> str, FunctionalProperty):
                """2D model of Well shape"""

            class hasShapeModel3D(Labware >> str, FunctionalProperty):
                """3D model of Well shape"""

            


In [ ]:
olw = labop_Labware()
olw.define_ontology()

list(olw.osolw.classes())

## Defining individuals

In [ ]:
with olw.osolw:
    greiner_384_v = olw.osolw.Labware("Greiner_384_V", 
                                        hasNumRows=192, 
                                        hasNumCols=16, 
                                        hasNumWells=384 )

In [ ]:

greiner_384_v.is_a


In [ ]:
type(greiner_384_v)

In [ ]:
greiner_384_v.hasNumCols, greiner_384_v.hasNumWells

In [ ]:
# loading an example labware catalog csv file:

print(os.getcwd())

strateos_csv = "./strateos_containers.csv"
strateos_cont_df = pd.read_csv(strateos_csv, delimiter=";")
strateos_cont_df = strateos_cont_df.reset_index()  # make sure indexes pair with number of rows
strateos_cont_df.head(32)

In [ ]:
strateos_cont_df[strateos_cont_df['Vendor'] == 'Greiner']

In [ ]:
with olw.osolw:
  for index,row in strateos_cont_df.iterrows():
    print(row['Id'], "-- >", row['Well Count'])
    lw = olw.osolw.Labware( row['Id'],
                             hasManifacturer=row['Vendor'],
                             hasNumRows=row['Well Count'] / row['Column Count'], 
                             hasNumCols=row['Column Count'],
                             hasNumWells=row['Well Count'],
                             hasHeight=row['Height (mm)'],
                             hasWellVolume=row['Well Volume (ul)'],
                               )

### exporting ontology as ttl file

## SPARQL queries

In [ ]:
prefix_dict = {
    'rdf': "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    'rdfs': "http://www.w3.org/2000/01/rdf-schema#",
    'xml': "http://www.w3.org/XML/1998/namespace",
    'xsd': "http://www.w3.org/2001/XMLSchema#",
    'owl': "http://www.w3.org/2002/07/owl#",
    'skos': "http://www.w3.org/2004/02/skos/core#",
    'dc': "http://purl.org/dc/elements/1.1/",
    'dcterm': "http://purl.org/dc/terms/",
    'dctype': "http://purl.org/dc/dcmitype/",
    'foaf': "http://xmlns.com/foaf/0.1/",
    'wd': "http://www.wikidata.org/entity/",
    'ex': "http://www.example.com/",
    'emmo': "http://emmo.info/emmo#",
    'oso': "http://www.oso.org/oso#",
    'osom': "http://www.oso.org/oso/measurements#",
    'osolw': "http://www.oso.org/oso/labware#",
}

In [ ]:
graph = olw.emmo_world.as_rdflib_graph()

for prefix, iri in prefix_dict.items():
    print(prefix, "--- ", iri )
    graph.bind(prefix, iri)

In [ ]:
# get all labware from Greiner

query = """

PREFIX osolw: <http://www.oso.org/oso/labware#>

SELECT ?lm 
WHERE {
    ?lm rdf:type osolw:Labware.
    ?lm osolw:hasManifacturer "Greiner".
    }
"""

In [ ]:
results = list(olw.emmo_world.sparql(query))
results

In [ ]:
# get all labware from Greiner with 384 wells 

query = """

PREFIX osolw: <http://www.oso.org/oso/labware#>

SELECT ?lm 
WHERE {
    ?lm rdf:type osolw:Labware.
    ?lm osolw:hasManifacturer "Greiner".
    ?lm osolw:hasNumWells 384.
    }
"""

In [ ]:
results = list(olw.emmo_world.sparql(query))
results

In [ ]:
results = list(olw.emmo_world.sparql(query))
results

## saving ontology as ttl file

In [ ]:

# Save new ontology as owl
olw.osolw.sync_attributes(name_policy='uuid', class_docstring='elucidation',
                     name_prefix='labop_')
 
olw.osolw.set_version(version_iri=olw.labop_labware_version_iri)
olw.osolw.dir_label = False

olw.catalog_mappings[olw.labop_labware_version_iri] = olw.labop_labware_ttl_filename 

#################################################################
# Annotate the ontology metadata
#################################################################

olw.osolw.metadata.abstract.append(en(
        'An EMMO-based domain ontology scientific measurements.'
        'olw-measurement is released under the Creative Commons Attribution 4.0 '
        'International license (CC BY 4.0).'))


olw.osolw.metadata.title.append(en('OSO-Measurement'))
olw.osolw.metadata.creator.append(en('mark doerr'))
olw.osolw.metadata.contributor.append(en('university greifswald'))
olw.osolw.metadata.publisher.append(en(''))
olw.osolw.metadata.license.append(en(
    'https://creativecommons.org/licenses/by/4.0/legalcode'))
olw.osolw.metadata.versionInfo.append(en(olw.__version__))
olw.osolw.metadata.comment.append(en(
    'The EMMO requires FaCT++ reasoner plugin in order to visualize all'
    'inferences and class hierarchy (ctrl+R hotkey in Protege).'))
olw.osolw.metadata.comment.append(en(
    'This ontology is generated with data from the ASE Python package.'))
olw.osolw.metadata.comment.append(en(
    'Contacts:\n'
    'mark doerr\n'
    'University Greifswald\n'
    'email: mark.doerr@suni-greifswald.de\n'
    '\n'
    ))

olw.osolw.save(olw.labop_labware_ttl_filename , overwrite=True)
#olw.save(labop_measurement_owl_filename, overwrite=True)
write_catalog(olw.catalog_mappings)
# olw.sync_reasoner()
# olw.save('olw-measurement-inferred.ttl', overwrite=True)
# ...and to the sqlite3 database.
# world.save()


# Manually change url of EMMO to `emmo_url` when importing it to make
# it resolvable without consulting the catalog file.  This makes it possible
# to open the ontology from url in Protege
import rdflib  # noqa: E402, F401
g = rdflib.Graph()
g.parse(olw.labop_labware_ttl_filename , format='turtle')
for s, p, o in g.triples(
        (None, rdflib.URIRef('http://www.w3.org/2002/07/owl#imports'), None)):
    if 'emmo-inferred' in o:
        g.remove((s, p, o))
        g.add((s, p, rdflib.URIRef(olw.emmo_url)))
g.serialize(destination=olw.labop_labware_ttl_filename, format='turtle')

